## 1. Set Up

In [ ]:
!pip install -q sentencepiece
!pip install -q transformers
!pip install -q datasets
!pip install pydot --quiet

     |████████████████████████████████| 1.3 MB 14.2 MB/s 
     |████████████████████████████████| 5.5 MB 11.6 MB/s 
     |████████████████████████████████| 182 kB 53.8 MB/s 
     |████████████████████████████████| 7.6 MB 72.7 MB/s 
     |████████████████████████████████| 451 kB 14.4 MB/s 
     |████████████████████████████████| 212 kB 89.3 MB/s 
     |████████████████████████████████| 115 kB 88.8 MB/s 
     |████████████████████████████████| 127 kB 90.0 MB/s 


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git -q
# !wget -N https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip . -q
# !unzip -q -n BLEURT-20.zip

!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D12.zip . -q
!unzip -q -n BLEURT-20-D12.zip

     |████████████████████████████████| 352 kB 15.0 MB/s 


In [ ]:
import re
import os
import sklearn as sk
import numpy as np

import pandas as pd
import ast
import matplotlib.pyplot as plt

import torch  # Only if you use a pytorch model, both options are shown below
from transformers import M2M100Config, M2M100Tokenizer, M2M100ForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

import nltk
from nltk.data import find

import tensorflow as tf
from tensorflow import keras

import datasets
from datasets import list_datasets, list_metrics, load_dataset, load_metric
from pprint import pprint
from bleurt import score
import csv
import random

#### 1.1 M2M100

In [ ]:
MODEL_CHECKPOINT = 'facebook/m2m100_418M'
m2m_tokenizer = M2M100Tokenizer.from_pretrained(MODEL_CHECKPOINT)
m2m_model = M2M100ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
# bleurt_metric = score.BleurtScorer(BLEURT_CHECKPOINT)

Downloading:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/272 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls drive/MyDrive

Mounted at /content/drive
'bert2bert_en_zh (1).ipynb'   Final_project_FJ.ipynb	  W266
'Colab Notebooks'	      MBart_Mini_Tuner.ipynb
 Final_project_FJ_2.ipynb     Text_classification.ipynb


In [ ]:
# bleu_metric = datasets.load_metric("bleu")
bleurt_metric = datasets.load_metric('bleurt', 'bleurt-20-d12')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


In [ ]:
random.seed(55)

## 2. Baseline

### 2.1 Streaming Method

In [ ]:
MODEL_CHECKPOINT = 'facebook/m2m100_418M'


PAIRS = [    
    'en-zh',
    'zh-en',
    'en-es',
    'es-zh',
    'es-en',
    'zh-es',
    ]
m2m_DATA = {
     'en-zh': {
        'size': 69020,
        'src': 'en',
        'tgt': 'zh',
        'tkn': 'zh_CN',
        },
    'zh-en': {
        'size': 69020,
        'src': 'zh',
        'tgt': 'en',
        'tkn': 'en_XX',
        },
    'en-es': {
        'size': 238511,
        'src': 'en',
        'tgt': 'es',
        'tkn': 'es_XX',
        },
    'es-zh': {
        'size': 65408,
        'src': 'es',
        'tgt': 'zh',
        'tkn': 'zh_CN',
        },
    'es-en': {
        'size': 238511,
        'src': 'es',
        'tgt': 'en',
        'tkn': 'en_XX',
        },
    'zh-es': {
        'size': 65408,
        'src': 'zh',
        'tgt': 'es',
        'tkn': 'es_XX',
        },
    }

DATASET = 'news_commentary'
MAX_LENGTH = 50
MAX_NEW_TOKENS = 50
TRUNCATION = True
PADDING = True
RETURN_TENSORS = "pt"
# BLEURT_CHECKPOINT = "./BLEURT-20"
N_EXAMPLES = 200

In [ ]:
# m2m_tokenizer = M2M100Tokenizer.from_pretrained(MODEL_CHECKPOINT)
# m2m_model = M2M100ForConditionalGeneration.from_pretrained(MODEL_CHECKPOINT)
bleurt_metric = score.BleurtScorer(BLEURT_CHECKPOINT)

NameError: ignored

In [ ]:
shuffled =  load_dataset('news_commentary',
                    'en-es',
                    streaming=True)

In [ ]:
shuffled['train']

In [ ]:
#Function to generate dataset stream, select first n samples, generate predictions and return score and n samples used 
def score_dataset(pair_index=0, starting_iter=1, end_iter = 9999, n_examples=N_EXAMPLES, 
                  dataset=DATASET, model=m2m_model, tokenizer=m2m_tokenizer, bleurt_metric=bleurt_metric):
    model = model
    tokenizer = tokenizer
    bleurt_metric = bleurt_metric
    pair_score = []

    pair = PAIRS[pair_index]
    src = m2m_DATA[PAIRS[pair_index]]['src']
    tgt = m2m_DATA[PAIRS[pair_index]]['tgt']
    # tkn = m2m_DATA[PAIRS[pair_index]]['tkn']
    size = m2m_DATA[PAIRS[pair_index]]['size']
    eval = size * 0.15
    loops = int(eval // n_examples)-starting_iter
    extra = int(round(eval % n_examples))
    prev_iter = 0
    curr_iter = prev_iter + 1

    if end_iter == 9999:
      end_iter = loops
    else:
      loops = end_iter - starting_iter

    tokenizer.src_lang = src
    tokenizer.tgt_lang = tgt

    shuffled =  load_dataset(DATASET,
                    pair,
                    streaming=True)
    
    with open(f'{pair}.csv', 'a', newline='') as myfile:
      wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
      wr.writerow([f'starting iter {prev_iter} ending at {end_iter}'])
    
    if (starting_iter > 0):
      prev_iter = starting_iter
    
    for run in range(loops+1):
      start = prev_iter * n_examples

      if run == loops:
        score_set = shuffled['train'].skip(start).take(extra)
      else:
        score_set = shuffled['train'].skip(start).take(n_examples)

      src_data = [p['translation'][src] for p in score_set]
      tgt_data = [p['translation'][tgt] for p in score_set]

      print(f'Now scoring iteration {run+1} of {loops+1}.')
      inputs = tokenizer(src_data,
                        padding=PADDING,
                        return_tensors=RETURN_TENSORS,
                        truncation=TRUNCATION,
                        max_length=MAX_LENGTH)
      
      generated_tokens = m2m_model.generate(
                        **inputs,
                        forced_bos_token_id=m2m_tokenizer.get_lang_id(tgt))
      
      generated_translations = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      actual_translations = tgt_data   

      bleurt_score = bleurt_metric.score(references=actual_translations, candidates=generated_translations)
      pair_score.append(bleurt_score)
      with open(f'{pair}.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(bleurt_score)
      prev_iter += 1

    return pair_score

In [ ]:
scores = dict()
for p in range(3,len(PAIRS)):
  print (f'Evaluating translation {PAIRS[p]}')
  scores[PAIRS[p]] = dict()
  scores[PAIRS[p]]['bleurt_score'] = score_dataset(
      pair_index=p, 
      starting_iter=0,
      n_examples=N_EXAMPLES,
      dataset=DATASET,
      model=m2m_model, 
      tokenizer=m2m_tokenizer, 
      bleurt_metric=bleurt_metric)

### 2.2 Loop Method

In [ ]:
dataset = load_dataset('news_commentary','en-zh')

Generating train split:   0%|          | 0/69206 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/en-zh/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 69206
    })
})


In [ ]:
en_zh_baseline = load_dataset('news_commentary','en-zh',split='train[:15%]')['translation']
baseline_file = '/content/sample_data/en_zh_baseline.csv'
pd.DataFrame(en_zh_baseline).to_csv(baseline_file)
df_baseline = pd.read_csv(baseline_file)

In [ ]:
df_baseline_list = df_baseline['en'].tolist()
df_baseline_label = df_baseline['zh'].tolist()

In [ ]:
df_baseline_label[:5]

['1929年还是1989年?',
 '巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。',
 '如今人们的心情却是沉重多了，许多人开始把这次危机与1929年和1931年相比，即使一些国家政府的表现仍然似乎把视目前的情况为是典型的而看见的衰退。目前的趋势是，要么是过度的克制（欧洲），要么是努力的扩展（美国）。欧洲在避免债务和捍卫欧元的名义下正变得谨慎，而美国已经在许多方面行动起来，以利用这一理想的时机来实行急需的结构性改革。',
 '然而，作为地域战略学家，无论是从政治意义还是从经济意义上，让我自然想到的年份是1989年。当然，雷曼兄弟公司的倒闭和柏林墙的倒塌没有任何关系。事实上，从表面上看，两者似乎是完全是相反的：一个是象征着压抑和人为分裂的柏林墙的倒塌，而另一个是看似坚不可摧的并令人安心的金融资本主义机构的倒塌。',
 '然而，和1989年一样，2008-2009年很可能也能被视为一个划时代的改变，其带来的发人深省的后果将在几十年后仍能让我们感受得到。东西方意识形态鸿沟的结束，以及对市场绝对信心的后果，都是历史的转折点。而2009年所发生的事情可能会威胁1989年革命所带来的积极成果，包括欧洲的和平统一和民主制度战胜了民族主义倾向，如果不是恐外倾向的话。']

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_baseline_list),step):
  curr_input = df_baseline_list[row_index: row_index + step]
  curr_label = df_baseline_label[row_index: row_index + step]

  curr_loop = row_index/step
  print("Begin current loop:", curr_loop)

  encoded_en = m2m_tokenizer(curr_input, 
                        padding = True,
                        max_length=50,
                        truncation=True,
                        return_tensors='pt')
  generated_tokens = m2m_model.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("zh"))
  curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

  ### Compute BLEU score
  # bleu_score = []
  # for sentence, label in zip(curr_output, curr_label):
  #   ### BLEU - Convert decoded into list
  #   curr_predict = [sentence.split()]
  #   curr_label_list = [[label.split()]]

  #   curr_bleu_score = bleu_metric.compute(predictions=curr_predict, references=curr_label_list)
  
  #   bleu_score.append(curr_bleu_score['bleu'])

  ### Compute BLEURT score
  bleurt_label = [[label] for label in curr_label]
  curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

  eval_score.append(curr_bleurt_score['scores'])

  
  df_output = pd.DataFrame(eval_score)
  df_output.to_csv('drive/MyDrive/W266/en_zh_bleurt_distilled_output.csv', sep=',', index=False)
  

Begin current loop: 0.0
Begin current loop: 1.0
Begin current loop: 2.0
Begin current loop: 3.0
Begin current loop: 4.0
Begin current loop: 5.0
Begin current loop: 6.0
Begin current loop: 7.0
Begin current loop: 8.0
Begin current loop: 9.0
Begin current loop: 10.0
Begin current loop: 11.0
Begin current loop: 12.0
Begin current loop: 13.0
Begin current loop: 14.0
Begin current loop: 15.0
Begin current loop: 16.0
Begin current loop: 17.0
Begin current loop: 18.0
Begin current loop: 19.0
Begin current loop: 20.0
Begin current loop: 21.0
Begin current loop: 22.0
Begin current loop: 23.0
Begin current loop: 24.0
Begin current loop: 25.0
Begin current loop: 26.0
Begin current loop: 27.0
Begin current loop: 28.0
Begin current loop: 29.0
Begin current loop: 30.0
Begin current loop: 31.0
Begin current loop: 32.0
Begin current loop: 33.0
Begin current loop: 34.0
Begin current loop: 35.0
Begin current loop: 36.0
Begin current loop: 37.0
Begin current loop: 38.0
Begin current loop: 39.0
Begin curr

## 3. Fine Tune Model

### 3.1 ZH-ES

In [ ]:
zh_es_train = load_dataset('news_commentary','es-zh',split='train[:70%]')['translation']
zh_es_validation = load_dataset('news_commentary','es-zh',split='train[70%:]')['translation']
zh_es_val = random.sample(zh_es_validation,5000)
# zh_es_test = load_dataset('news_commentary','es-zh',split='train[120:140]')['translation']

In [ ]:
train_file = 'drive/MyDrive/W266/zh_es_train.csv'
valid_file = 'drive/MyDrive/W266/zh_es_val.csv'
# test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(zh_es_train).to_csv(train_file)
pd.DataFrame(zh_es_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv(train_file)
df_valid = pd.read_csv(valid_file)
# df_test = pd.read_csv(test_file)

In [ ]:
def preprocess_data(text_pair, tokenizer, max_length=50):
    # For reversed sequence
    target_text, orig_text = text_pair
    
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
# For normal sequence 
# def preprocess_data(text_pair, tokenizer, max_length=50):
#     orig_text, target_text = text_pair
#     orig_encoded = tokenizer.batch_encode_plus(
#         [orig_text],
#         max_length=max_length,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors='pt'
#     )

#     # not sure if the below two lines are needed for m2m100
#     orig_input_ids = orig_encoded['input_ids'][0]
#     orig_attention_mask = orig_encoded['attention_mask'][0]
    
#     target_encoded = tokenizer.batch_encode_plus(
#         [target_text],
#         max_length=max_length,
#         padding='max_length',
#         truncation=True,
#         return_attention_mask=True,
#         return_tensors='pt'
#     )
    
#     label_ids = target_encoded['input_ids'][0]
    
#     return {'input_ids': orig_input_ids,
#             'attention_mask': orig_attention_mask,
#             'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pair = df[['es', 'zh']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(zh_es_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(zh_es_validation),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'zh'
tgt_lang = 'es'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 32

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 45797
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1432
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,1.130400,1.028982


Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-500
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1000
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 19627
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1432, training_loss=1.2718614546280333, metrics={'train_runtime': 8081.7575, 'train_samples_per_second': 5.667, 'train_steps_per_second': 0.177, 'total_flos': 1.2405861182865408e+16, 'train_loss': 1.2718614546280333, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_es/checkpoint-1/pytorch_model.bin


In [ ]:
for test_input_text in ['萨尔科奇的宝莱坞新娘',
                        '对理性的抵制']:
    test_inputs = m2m_tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = m2m_model.generate(test_inputs['input_ids'].cuda(),
                                         forced_bos_token_id=m2m_tokenizer.get_lang_id("es"))

    print([m2m_tokenizer.batch_decode(test_output_ids, skip_special_tokens=True)])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[['La novia de Bollywood de Sarkozy']]
[['Resistencia a la racionalidad']]


In [ ]:
m2m_model_saved = M2M100ForConditionalGeneration.from_pretrained(file_path + '/checkpoint-1')

In [ ]:
for test_input_text in ['萨尔科奇的宝莱坞新娘',
                        '对理性的抵制']:
    test_inputs = m2m_tokenizer([test_input_text], return_tensors='pt')
    test_output_ids = m2m_model_saved.generate(test_inputs['input_ids'],
                                         forced_bos_token_id=m2m_tokenizer.get_lang_id("es"))

    print([m2m_tokenizer.batch_decode(test_output_ids, skip_special_tokens=True)])

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


[['La novia de Bollywood de Sarkozy']]
[['Resistencia a la racionalidad']]


In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_valid_list),step):
  curr_input = df_valid_list[row_index: row_index + step]
  curr_label = df_valid_label[row_index: row_index + step]

  curr_loop = row_index/step
  print("Begin current loop:", curr_loop)

  encoded_en = m2m_tokenizer(curr_input, 
                        padding = True,
                        max_length=50,
                        truncation=True,
                        return_tensors='pt')
  generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
  curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

  ### Compute BLEURT score
  bleurt_label = [[label] for label in curr_label]
  curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

  eval_score.append(curr_bleurt_score['scores'])

  
  df_output = pd.DataFrame(eval_score)
  df_output.to_csv('drive/MyDrive/W266/zh_es_bleurt_validation.csv', sep=',', index=False)
  

Begin current loop: 0.0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Begin current loop: 1.0
Begin current loop: 2.0
Begin current loop: 3.0
Begin current loop: 4.0
Begin current loop: 5.0
Begin current loop: 6.0
Begin current loop: 7.0
Begin current loop: 8.0
Begin current loop: 9.0
Begin current loop: 10.0
Begin current loop: 11.0
Begin current loop: 12.0
Begin current loop: 13.0
Begin current loop: 14.0
Begin current loop: 15.0
Begin current loop: 16.0
Begin current loop: 17.0
Begin current loop: 18.0
Begin current loop: 19.0
Begin current loop: 20.0
Begin current loop: 21.0
Begin current loop: 22.0
Begin current loop: 23.0
Begin current loop: 24.0
Begin current loop: 25.0
Begin current loop: 26.0
Begin current loop: 27.0
Begin current loop: 28.0
Begin current loop: 29.0
Begin current loop: 30.0
Begin current loop: 31.0
Begin current loop: 32.0
Begin current loop: 33.0
Begin current loop: 34.0
Begin current loop: 35.0
Begin current loop: 36.0
Begin current loop: 37.0
Begin current loop: 38.0
Begin current loop: 39.0
Begin current loop: 40.0
Begin cur

### 3.2 ES-ZH

In [ ]:
es_zh_train = load_dataset('news_commentary','es-zh',split='train[:70%]')['translation']
es_zh_validation = load_dataset('news_commentary','es-zh',split='train[70%:]')['translation']
es_zh_val = random.sample(es_zh_validation,5000)

Generating train split:   0%|          | 0/65424 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/es-zh/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


In [ ]:
train_file = 'drive/MyDrive/W266/es_zh_train.csv'
valid_file = 'drive/MyDrive/W266/es_zh_val.csv'
# test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(es_zh_train).to_csv(train_file)
pd.DataFrame(es_zh_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv(train_file)
df_valid = pd.read_csv(valid_file)
# df_test = pd.read_csv(test_file)

In [ ]:
# For normal sequence 
def preprocess_data(text_pair, tokenizer, max_length=50):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # not sure if the below two lines are needed for m2m100
    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        # check the text pair
        text_pair = df[['es', 'zh']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(es_zh_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(es_zh_val),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'es'
tgt_lang = 'zh'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 5

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 45797
  Num Epochs = 1
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 9160
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,1.708900,1.688322


Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-500
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-1000
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-1500
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-1500/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-2000
Configuration saved in driv

TrainOutput(global_step=9160, training_loss=1.8592834705869183, metrics={'train_runtime': 13294.0811, 'train_samples_per_second': 3.445, 'train_steps_per_second': 0.689, 'total_flos': 1.2405861182865408e+16, 'train_loss': 1.8592834705869183, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-001')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-001
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-001/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-001/pytorch_model.bin


In [ ]:
m2m_model_es_zh = M2M100ForConditionalGeneration.from_pretrained(file_path + '/checkpoint-001')

loading configuration file drive/MyDrive/W266/M2M100_news_commentary_es_zh/checkpoint-001/config.json
Model config M2M100Config {
  "_name_or_path": "facebook/m2m100_418M",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "M2M100ForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.05,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.05,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "m2m_100",
  "num_beams": 5,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": true,
  "torch_dtype": "float32",
  "transformers_version

In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_valid_list),step):
  curr_input = df_valid_list[row_index: row_index + step]
  curr_label = df_valid_label[row_index: row_index + step]

  curr_loop = row_index/step
  print("Begin current loop:", curr_loop)

  encoded_en = m2m_tokenizer(curr_input, 
                        padding = True,
                        max_length=50,
                        truncation=True,
                        return_tensors='pt')
  generated_tokens = m2m_model_es_zh.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
  curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

  ### Compute BLEURT score
  bleurt_label = [[label] for label in curr_label]
  curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

  eval_score.append(curr_bleurt_score['scores'])

  
  df_output = pd.DataFrame(eval_score)
  df_output.to_csv('drive/MyDrive/W266/es_zh_bleurt_validation.csv', sep=',', index=False)
  

Begin current loop: 0.0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Begin current loop: 1.0
Begin current loop: 2.0
Begin current loop: 3.0
Begin current loop: 4.0
Begin current loop: 5.0
Begin current loop: 6.0
Begin current loop: 7.0
Begin current loop: 8.0
Begin current loop: 9.0
Begin current loop: 10.0
Begin current loop: 11.0
Begin current loop: 12.0
Begin current loop: 13.0
Begin current loop: 14.0
Begin current loop: 15.0
Begin current loop: 16.0
Begin current loop: 17.0
Begin current loop: 18.0
Begin current loop: 19.0
Begin current loop: 20.0
Begin current loop: 21.0
Begin current loop: 22.0
Begin current loop: 23.0
Begin current loop: 24.0
Begin current loop: 25.0
Begin current loop: 26.0
Begin current loop: 27.0
Begin current loop: 28.0
Begin current loop: 29.0
Begin current loop: 30.0
Begin current loop: 31.0
Begin current loop: 32.0
Begin current loop: 33.0
Begin current loop: 34.0
Begin current loop: 35.0
Begin current loop: 36.0
Begin current loop: 37.0
Begin current loop: 38.0
Begin current loop: 39.0
Begin current loop: 40.0
Begin cur

### 3.3 ZH-EN

In [ ]:
zh_en_train = load_dataset('news_commentary','en-zh',split='train[:70%]')['translation']
zh_en_validation = load_dataset('news_commentary','en-zh',split='train[70%:]')['translation']
zh_en_val = random.sample(zh_en_validation,5000)
# zh_es_test = load_dataset('news_commentary','es-zh',split='train[120:140]')['translation']

Generating train split:   0%|          | 0/69206 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/en-zh/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


In [ ]:
train_file = 'drive/MyDrive/W266/zh_en_train.csv'
valid_file = 'drive/MyDrive/W266/zh_en_val.csv'
# test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(zh_en_train).to_csv(train_file)
pd.DataFrame(zh_en_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv(train_file)
df_valid = pd.read_csv(valid_file)
# df_test = pd.read_csv(test_file)

In [ ]:
def preprocess_data(text_pair, tokenizer, max_length=50):
    # For reversed sequence
    target_text, orig_text = text_pair
    
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pair = df[['en', 'zh']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

#### 3.3.1 Baseline

In [ ]:
df_baseline_list = df_valid['zh'].tolist()
df_baseline_label = df_valid['en'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_baseline_list),step):
  curr_input = df_baseline_list[row_index: row_index + step]
  curr_label = df_baseline_label[row_index: row_index + step]

  curr_loop = row_index/step
  print("Begin current loop:", curr_loop)

  encoded_en = m2m_tokenizer(curr_input, 
                        padding = True,
                        max_length=50,
                        truncation=True,
                        return_tensors='pt')
  generated_tokens = m2m_model.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("en"))
  curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

  ### Compute BLEURT score
  bleurt_label = [[label] for label in curr_label]
  curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

  eval_score.append(curr_bleurt_score['scores'])

  
  df_output = pd.DataFrame(eval_score)
  df_output.to_csv('drive/MyDrive/W266/zh_en_bleurt_baseline.csv', sep=',', index=False)

Begin current loop: 0.0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Begin current loop: 1.0
Begin current loop: 2.0
Begin current loop: 3.0
Begin current loop: 4.0
Begin current loop: 5.0
Begin current loop: 6.0
Begin current loop: 7.0
Begin current loop: 8.0
Begin current loop: 9.0
Begin current loop: 10.0
Begin current loop: 11.0
Begin current loop: 12.0
Begin current loop: 13.0
Begin current loop: 14.0
Begin current loop: 15.0
Begin current loop: 16.0
Begin current loop: 17.0
Begin current loop: 18.0
Begin current loop: 19.0
Begin current loop: 20.0
Begin current loop: 21.0
Begin current loop: 22.0
Begin current loop: 23.0
Begin current loop: 24.0
Begin current loop: 25.0
Begin current loop: 26.0
Begin current loop: 27.0
Begin current loop: 28.0
Begin current loop: 29.0
Begin current loop: 30.0
Begin current loop: 31.0
Begin current loop: 32.0
Begin current loop: 33.0
Begin current loop: 34.0
Begin current loop: 35.0
Begin current loop: 36.0
Begin current loop: 37.0
Begin current loop: 38.0
Begin current loop: 39.0
Begin current loop: 40.0
Begin cur

#### 3.3.2 Fine tune model

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(zh_en_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(zh_en_val),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'zh'
tgt_lang = 'en'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 8

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 48444
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6056
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,0.934200,0.904434


Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-500
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-1000
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-1500
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-1500/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-2000
Configuration saved in driv

TrainOutput(global_step=6056, training_loss=1.0712783698834076, metrics={'train_runtime': 12239.1281, 'train_samples_per_second': 3.958, 'train_steps_per_second': 0.495, 'total_flos': 1.3122901918089216e+16, 'train_loss': 1.0712783698834076, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-001')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-001
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-001/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-001/pytorch_model.bin


In [ ]:
m2m_model_saved = M2M100ForConditionalGeneration.from_pretrained(file_path + '/checkpoint-001')

loading configuration file drive/MyDrive/W266/M2M100_news_commentary_zh_en/checkpoint-001/config.json
Model config M2M100Config {
  "_name_or_path": "facebook/m2m100_418M",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "M2M100ForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.05,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.05,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "m2m_100",
  "num_beams": 5,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": true,
  "torch_dtype": "float32",
  "transformers_version

In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
len(df_valid_list)

5000

In [ ]:
eval_score = []
step = 100


for row_index in range(700,len(df_valid_list),step):

  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/zh_en_bleurt_validation_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_valid_list[row_index: row_index + step]
    curr_label = df_valid_label[row_index: row_index + step]


    # curr_loop = row_index/step
    print(f"  Processing row_index: {row_index}")

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/zh_en_bleurt_validation_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 700
  Processing row_index: 700


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 800
  Processing row_index: 800
Processing row_index: 900
  Processing row_index: 900
Processing row_index: 1000
  Processing row_index: 1000
Processing row_index: 1100
  Processing row_index: 1100
Processing row_index: 1200
  Processing row_index: 1200
Processing row_index: 1300
  Processing row_index: 1300
Processing row_index: 1400
  Processing row_index: 1400
Processing row_index: 1500
  Processing row_index: 1500
Processing row_index: 1600
  Processing row_index: 1600
Processing row_index: 1700
  Processing row_index: 1700
Processing row_index: 1800
  Processing row_index: 1800
Processing row_index: 1900
  Processing row_index: 1900
Processing row_index: 2000
  Processing row_index: 2000
Processing row_index: 2100
  Processing row_index: 2100
Processing row_index: 2200
  Processing row_index: 2200
Processing row_index: 2300
  Processing row_index: 2300
Processing row_index: 2400
  Processing row_index: 2400
Processing row_index: 2500
  Processing row_index: 2

### 3.4 EN-ZH

In [ ]:
en_zh_train = load_dataset('news_commentary','en-zh',split='train[:70%]')['translation']
en_zh_validation = load_dataset('news_commentary','en-zh',split='train[70%:]')['translation']
en_zh_val = random.sample(en_zh_validation,5000)
# zh_es_test = load_dataset('news_commentary','es-zh',split='train[120:140]')['translation']

Generating train split:   0%|          | 0/69206 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/en-zh/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


In [ ]:
train_file = 'drive/MyDrive/W266/en_zh_train.csv'
valid_file = 'drive/MyDrive/W266/en_zh_val.csv'
# # test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(en_zh_train).to_csv(train_file)
pd.DataFrame(en_zh_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv('drive/MyDrive/W266/en_zh_train.csv')
df_valid = pd.read_csv('drive/MyDrive/W266/en_zh_val.csv')
# df_test = pd.read_csv(test_file)

In [ ]:
example_pair = df_valid[['en', 'zh']].values.astype(str)[8]
example_input, example_label = example_pair
example_input

'With the stakes so high, the Russian-US initiative to organize a peace conference has not only been welcomed, but is seen as an indispensable initiative that must be planned with excruciating care. We are likely to hear of delay after delay in the coming days, as various sides try to choose their representatives.'

In [ ]:
# For normal sequence 
def preprocess_data(text_pair, tokenizer, max_length=50):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # not sure if the below two lines are needed for m2m100
    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pair = df[['en', 'zh']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

#### 3.4.1 Baseline

In [ ]:
df_baseline_list = df_valid['en'].tolist()
df_baseline_label = df_valid['zh'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_baseline_list),step):
  
  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/en_zh/en_zh_baseline_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_baseline_list[row_index: row_index + step]
    curr_label = df_baseline_label[row_index: row_index + step]

    print("Begin current loop:", row_index)

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("zh"))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/en_zh/en_zh_baseline_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 0
Begin current loop: 0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 100
Begin current loop: 100
Processing row_index: 200
Begin current loop: 200
Processing row_index: 300
Begin current loop: 300
Processing row_index: 400
Begin current loop: 400
Processing row_index: 500
Begin current loop: 500
Processing row_index: 600
Begin current loop: 600
Processing row_index: 700
Begin current loop: 700
Processing row_index: 800
Begin current loop: 800
Processing row_index: 900
Begin current loop: 900
Processing row_index: 1000
Begin current loop: 1000
Processing row_index: 1100
Begin current loop: 1100
Processing row_index: 1200
Begin current loop: 1200
Processing row_index: 1300
Begin current loop: 1300
Processing row_index: 1400
Begin current loop: 1400
Processing row_index: 1500
Begin current loop: 1500
Processing row_index: 1600
Begin current loop: 1600
Processing row_index: 1700
Begin current loop: 1700
Processing row_index: 1800
Begin current loop: 1800
Processing row_index: 1900
Begin current loop: 1900
Processing row_index: 2000
Beg

#### 3.4.2 Fine tune model

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(en_zh_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(en_zh_val),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'en'
tgt_lang = 'zh'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/en_zh/new_model/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 8

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 48444
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6056
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,1.574800,1.564497


Saving model checkpoint to drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-500
Configuration saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1000
Configuration saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1500
Configuration saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1500/config.json
Model weights saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoi

TrainOutput(global_step=6056, training_loss=1.7123450262858342, metrics={'train_runtime': 11670.3874, 'train_samples_per_second': 4.151, 'train_steps_per_second': 0.519, 'total_flos': 1.3122901918089216e+16, 'train_loss': 1.7123450262858342, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001/pytorch_model.bin


In [ ]:
m2m_model_saved = M2M100ForConditionalGeneration.from_pretrained('drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001')

In [ ]:
m2m_tokenizer.src_lang = "en"
encoded_en = m2m_tokenizer(example_input, return_tensors="pt")
generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("zh"))
m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['因此,俄罗斯-美国组织和平会议的意图不但得到了欢迎,也被视为一个不可或缺的计划,必须以谨慎的态度进行规划。我们很可能会在未来几天中听到延迟之后的延迟,因为各方试图选择代表。']

In [ ]:
example_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
example_output

['因此,俄罗斯-美国组织和平会议的意图不但得到了欢迎,也被视为一个不可或缺的计划,必须以谨慎的态度进行规划。我们很可能会在未来几天中听到延迟之后的延迟,因为各方试图选择代表。']

In [ ]:
bleurt_label = [example_label]
bleurt_label

['因为利害攸关，俄美组织和平会议的计划不仅颇受欢迎，而且被视为不可或缺的行动而受到慎重对待。因为各方都力争选定代表，和平会议的日程可能一推再推。但我们必须保持耐心，因为会议一旦开始除成功外就没有别的选择。没有结果的努力不会有任何用处。']

In [ ]:
bleurt_metric.compute(predictions=example_output, references=bleurt_label)

{'scores': [0.29573166370391846]}

In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
eval_score = []
step = 100


for row_index in range(2500,len(df_valid_list),step):

  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/en_zh/new_model/zh_en_bleurt_validation_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_valid_list[row_index: row_index + step]
    curr_label = df_valid_label[row_index: row_index + step]


    # curr_loop = row_index/step
    print(f"  Processing row_index: {row_index}")

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/en_zh/new_model/zh_en_bleurt_validation_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 2500
  Processing row_index: 2500


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 2600
  Processing row_index: 2600
Processing row_index: 2700
  Processing row_index: 2700
Processing row_index: 2800
  Processing row_index: 2800
Processing row_index: 2900
  Processing row_index: 2900
Processing row_index: 3000
  Processing row_index: 3000
Processing row_index: 3100
  Processing row_index: 3100
Processing row_index: 3200
  Processing row_index: 3200
Processing row_index: 3300
  Processing row_index: 3300
Processing row_index: 3400
  Processing row_index: 3400
Processing row_index: 3500
  Processing row_index: 3500
Processing row_index: 3600
  Processing row_index: 3600
Processing row_index: 3700
  Processing row_index: 3700
Processing row_index: 3800
  Processing row_index: 3800
Processing row_index: 3900
  Processing row_index: 3900
Processing row_index: 4000
  Processing row_index: 4000
Processing row_index: 4100
  Processing row_index: 4100
Processing row_index: 4200
  Processing row_index: 4200
Processing row_index: 4300
  Processing row_inde

In [ ]:
en_zh_train = load_dataset('news_commentary','en-zh',split='train[:70%]')['translation']
en_zh_validation = load_dataset('news_commentary','en-zh',split='train[70%:]')['translation']
en_zh_val = random.sample(en_zh_validation,5000)
# zh_es_test = load_dataset('news_commentary','es-zh',split='train[120:140]')['translation']

Generating train split:   0%|          | 0/69206 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/en-zh/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


In [ ]:
train_file = 'drive/MyDrive/W266/en_zh_train.csv'
valid_file = 'drive/MyDrive/W266/en_zh_val.csv'
# # test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(en_zh_train).to_csv(train_file)
pd.DataFrame(en_zh_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv('drive/MyDrive/W266/en_zh_train.csv')
df_valid = pd.read_csv('drive/MyDrive/W266/en_zh_val.csv')
# df_test = pd.read_csv(test_file)

In [ ]:
example_pair = df_valid[['en', 'zh']].values.astype(str)[8]
example_input, example_label = example_pair
example_input

'With the stakes so high, the Russian-US initiative to organize a peace conference has not only been welcomed, but is seen as an indispensable initiative that must be planned with excruciating care. We are likely to hear of delay after delay in the coming days, as various sides try to choose their representatives.'

In [ ]:
# For normal sequence 
def preprocess_data(text_pair, tokenizer, max_length=50):
    orig_text, target_text = text_pair
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # not sure if the below two lines are needed for m2m100
    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pair = df[['en', 'zh']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

#### 3.4.1 Baseline

In [ ]:
df_baseline_list = df_valid['en'].tolist()
df_baseline_label = df_valid['zh'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_baseline_list),step):
  
  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/en_zh/en_zh_baseline_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_baseline_list[row_index: row_index + step]
    curr_label = df_baseline_label[row_index: row_index + step]

    print("Begin current loop:", row_index)

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("zh"))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/en_zh/en_zh_baseline_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 0
Begin current loop: 0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 100
Begin current loop: 100
Processing row_index: 200
Begin current loop: 200
Processing row_index: 300
Begin current loop: 300
Processing row_index: 400
Begin current loop: 400
Processing row_index: 500
Begin current loop: 500
Processing row_index: 600
Begin current loop: 600
Processing row_index: 700
Begin current loop: 700
Processing row_index: 800
Begin current loop: 800
Processing row_index: 900
Begin current loop: 900
Processing row_index: 1000
Begin current loop: 1000
Processing row_index: 1100
Begin current loop: 1100
Processing row_index: 1200
Begin current loop: 1200
Processing row_index: 1300
Begin current loop: 1300
Processing row_index: 1400
Begin current loop: 1400
Processing row_index: 1500
Begin current loop: 1500
Processing row_index: 1600
Begin current loop: 1600
Processing row_index: 1700
Begin current loop: 1700
Processing row_index: 1800
Begin current loop: 1800
Processing row_index: 1900
Begin current loop: 1900
Processing row_index: 2000
Beg

#### 3.4.2 Fine tune model

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(en_zh_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(en_zh_val),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'en'
tgt_lang = 'zh'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/en_zh/new_model/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 8

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 48444
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6056
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,1.574800,1.564497


Saving model checkpoint to drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-500
Configuration saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1000
Configuration saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1500
Configuration saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoint-1500/config.json
Model weights saved in drive/MyDrive/W266/en_zh/new_model/M2M100_news_commentary_en_zh/checkpoi

TrainOutput(global_step=6056, training_loss=1.7123450262858342, metrics={'train_runtime': 11670.3874, 'train_samples_per_second': 4.151, 'train_steps_per_second': 0.519, 'total_flos': 1.3122901918089216e+16, 'train_loss': 1.7123450262858342, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001/pytorch_model.bin


In [ ]:
m2m_model_saved = M2M100ForConditionalGeneration.from_pretrained('drive/MyDrive/W266/M2M100_news_commentary_en_zh/checkpoint-001')

In [ ]:
m2m_tokenizer.src_lang = "en"
encoded_en = m2m_tokenizer(example_input, return_tensors="pt")
generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("zh"))
m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['因此,俄罗斯-美国组织和平会议的意图不但得到了欢迎,也被视为一个不可或缺的计划,必须以谨慎的态度进行规划。我们很可能会在未来几天中听到延迟之后的延迟,因为各方试图选择代表。']

In [ ]:
example_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
example_output

['因此,俄罗斯-美国组织和平会议的意图不但得到了欢迎,也被视为一个不可或缺的计划,必须以谨慎的态度进行规划。我们很可能会在未来几天中听到延迟之后的延迟,因为各方试图选择代表。']

In [ ]:
bleurt_label = [example_label]
bleurt_label

['因为利害攸关，俄美组织和平会议的计划不仅颇受欢迎，而且被视为不可或缺的行动而受到慎重对待。因为各方都力争选定代表，和平会议的日程可能一推再推。但我们必须保持耐心，因为会议一旦开始除成功外就没有别的选择。没有结果的努力不会有任何用处。']

In [ ]:
bleurt_metric.compute(predictions=example_output, references=bleurt_label)

{'scores': [0.29573166370391846]}

In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
eval_score = []
step = 100


for row_index in range(2500,len(df_valid_list),step):

  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/en_zh/new_model/zh_en_bleurt_validation_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_valid_list[row_index: row_index + step]
    curr_label = df_valid_label[row_index: row_index + step]


    # curr_loop = row_index/step
    print(f"  Processing row_index: {row_index}")

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/en_zh/new_model/zh_en_bleurt_validation_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 2500
  Processing row_index: 2500


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 2600
  Processing row_index: 2600
Processing row_index: 2700
  Processing row_index: 2700
Processing row_index: 2800
  Processing row_index: 2800
Processing row_index: 2900
  Processing row_index: 2900
Processing row_index: 3000
  Processing row_index: 3000
Processing row_index: 3100
  Processing row_index: 3100
Processing row_index: 3200
  Processing row_index: 3200
Processing row_index: 3300
  Processing row_index: 3300
Processing row_index: 3400
  Processing row_index: 3400
Processing row_index: 3500
  Processing row_index: 3500
Processing row_index: 3600
  Processing row_index: 3600
Processing row_index: 3700
  Processing row_index: 3700
Processing row_index: 3800
  Processing row_index: 3800
Processing row_index: 3900
  Processing row_index: 3900
Processing row_index: 4000
  Processing row_index: 4000
Processing row_index: 4100
  Processing row_index: 4100
Processing row_index: 4200
  Processing row_index: 4200
Processing row_index: 4300
  Processing row_inde

### 3.5 EN-ES

In [ ]:
en_es_train = load_dataset('news_commentary','en-es',split='train[:50%]')['translation']
en_es_validation = load_dataset('news_commentary','en-es',split='train[50%:]')['translation']
en_es_val = random.sample(en_es_validation,5000)

Generating train split:   0%|          | 0/238872 [00:00<?, ? examples/s]

Dataset news_commentary downloaded and prepared to /root/.cache/huggingface/datasets/news_commentary/en-es/11.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4. Subsequent calls will reuse this data.


In [ ]:
train_file = 'drive/MyDrive/W266/en_es/en_es_train.csv'
valid_file = 'drive/MyDrive/W266/en_es/en_es_val.csv'
# test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(en_es_train).to_csv(train_file)
pd.DataFrame(en_es_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv(train_file)
df_valid = pd.read_csv(valid_file)
# df_test = pd.read_csv(test_file)

In [ ]:
df_train.shape

(119436, 3)

In [ ]:
def preprocess_data(text_pair, tokenizer, max_length=50):
    # For normal sequence
    orig_text,target_text = text_pair
    
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pair = df[['en', 'es']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

#### 3.5.1 Baseline

In [ ]:
df_baseline_list = df_valid['en'].tolist()
df_baseline_label = df_valid['es'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(4000,len(df_baseline_list),step):
  
  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/en_es/baseline/en_es_baseline_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_baseline_list[row_index: row_index + step]
    curr_label = df_baseline_label[row_index: row_index + step]

    print("Begin current loop:", row_index)

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("es"))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/en_es/baseline/en_es_baseline_{row_index}.csv', sep=',', index=False)

Processing row_index: 4000
Begin current loop: 4000


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 4100
Begin current loop: 4100
Processing row_index: 4200
Begin current loop: 4200
Processing row_index: 4300
Begin current loop: 4300
Processing row_index: 4400
Begin current loop: 4400
Processing row_index: 4500
Begin current loop: 4500
Processing row_index: 4600
Begin current loop: 4600
Processing row_index: 4700
Begin current loop: 4700
Processing row_index: 4800
Begin current loop: 4800
Processing row_index: 4900
Begin current loop: 4900


#### 3.5.2 Fine tune model

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(en_es_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(en_es_val),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'en'
tgt_lang = 'es'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/en_es/new_model/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 8

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 119436
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14930
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,0.298000,0.290281


Saving model checkpoint to drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-500
Configuration saved in drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-1000
Configuration saved in drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-1500
Configuration saved in drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoint-1500/config.json
Model weights saved in drive/MyDrive/W266/en_es/new_model/M2M100_news_commentary_en_es/checkpoi

TrainOutput(global_step=14930, training_loss=0.36821525251793524, metrics={'train_runtime': 20636.6756, 'train_samples_per_second': 5.788, 'train_steps_per_second': 0.723, 'total_flos': 3.2353788157231104e+16, 'train_loss': 0.36821525251793524, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_en_es/checkpoint-001')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_en_es/checkpoint-001
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_en_es/checkpoint-001/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_en_es/checkpoint-001/pytorch_model.bin


In [ ]:
m2m_model_saved = M2M100ForConditionalGeneration.from_pretrained('drive/MyDrive/W266/M2M100_news_commentary_en_es/checkpoint-001')

loading configuration file drive/MyDrive/W266/M2M100_news_commentary_en_es/checkpoint-001/config.json
Model config M2M100Config {
  "_name_or_path": "facebook/m2m100_418M",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "M2M100ForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.05,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.05,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "m2m_100",
  "num_beams": 5,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": true,
  "torch_dtype": "float32",
  "transformers_version

In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
eval_score = []
step = 100


for row_index in range(0,len(df_valid_list),step):

  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/en_es/new_model/en_es_validation_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_valid_list[row_index: row_index + step]
    curr_label = df_valid_label[row_index: row_index + step]


    # curr_loop = row_index/step
    print(f"  Processing row_index: {row_index}")

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/en_es/new_model/en_es_validation_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 0
  Processing row_index: 0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 100
  Processing row_index: 100
Processing row_index: 200
  Processing row_index: 200
Processing row_index: 300
  Processing row_index: 300
Processing row_index: 400
  Processing row_index: 400
Processing row_index: 500
  Processing row_index: 500
Processing row_index: 600
  Processing row_index: 600
Processing row_index: 700
  Processing row_index: 700
Processing row_index: 800
  Processing row_index: 800
Processing row_index: 900
  Processing row_index: 900
Processing row_index: 1000
  Processing row_index: 1000
Processing row_index: 1100
  Processing row_index: 1100
Processing row_index: 1200
  Processing row_index: 1200
Processing row_index: 1300
  Processing row_index: 1300
Processing row_index: 1400
  Processing row_index: 1400
Processing row_index: 1500
  Processing row_index: 1500
Processing row_index: 1600
  Processing row_index: 1600
Processing row_index: 1700
  Processing row_index: 1700
Processing row_index: 1800
  Processing row_index: 1800
Processing

### 3.6 ES-EN

In [ ]:
es_en_train = load_dataset('news_commentary','en-es',split='train[:50%]')['translation']
es_en_validation = load_dataset('news_commentary','en-es',split='train[50%:]')['translation']
es_en_val = random.sample(es_en_validation,5000)
# zh_es_test = load_dataset('news_commentary','es-zh',split='train[120:140]')['translation']

In [ ]:
train_file = 'drive/MyDrive/W266/es_en/es_en_train.csv'
valid_file = 'drive/MyDrive/W266/es_en/es_en_val.csv'
# # test_file = 'drive/MyDrive/W266/zh_es_test.csv'


pd.DataFrame(es_en_train).to_csv(train_file)
pd.DataFrame(es_en_val).to_csv(valid_file)
# pd.DataFrame(zh_es_test).to_csv(test_file)

In [ ]:
df_train = pd.read_csv('drive/MyDrive/W266/es_en/es_en_train.csv')
df_valid = pd.read_csv('drive/MyDrive/W266/es_en/es_en_val.csv')
# df_test = pd.read_csv(test_file)

In [ ]:
# For reverse sequence 
def preprocess_data(text_pair, tokenizer, max_length=50):
    target_text, orig_text = text_pair
    
    orig_encoded = tokenizer.batch_encode_plus(
        [orig_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # not sure if the below two lines are needed for m2m100
    orig_input_ids = orig_encoded['input_ids'][0]
    orig_attention_mask = orig_encoded['attention_mask'][0]
    
    target_encoded = tokenizer.batch_encode_plus(
        [target_text],
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    label_ids = target_encoded['input_ids'][0]
    
    return {'input_ids': orig_input_ids,
            'attention_mask': orig_attention_mask,
            'labels': label_ids}

In [ ]:
class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        row_to_load = self.row_order[idx]
        df = pd.read_csv(self.data_filename,
                         skiprows=range(1, row_to_load),
                         nrows=1)
        
        text_pair = df[['en', 'es']].values.astype(str)[0]
        
        item_data = preprocess_data(
            text_pair,
            self.tokenizer,
            self.max_length
        )

        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

#### 3.6.1 Baseline

In [ ]:
df_baseline_list = df_valid['es'].tolist()
df_baseline_label = df_valid['en'].tolist()

In [ ]:
eval_score = []
step = 100

for row_index in range(0,len(df_baseline_list),step):
  
  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/es_en/baseline/es_en_base_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_baseline_list[row_index: row_index + step]
    curr_label = df_baseline_label[row_index: row_index + step]

    print("Begin current loop:", row_index)

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id("en"))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/es_en/baseline/es_en_base_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 0
Begin current loop: 0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Processing row_index: 100
Begin current loop: 100
Processing row_index: 200
Begin current loop: 200
Processing row_index: 300
Begin current loop: 300
Processing row_index: 400
Begin current loop: 400
Processing row_index: 500
Begin current loop: 500
Processing row_index: 600
Begin current loop: 600
Processing row_index: 700
Begin current loop: 700
Processing row_index: 800
Begin current loop: 800
Processing row_index: 900
Begin current loop: 900
Processing row_index: 1000
Begin current loop: 1000
Processing row_index: 1100
Begin current loop: 1100
Processing row_index: 1200
Begin current loop: 1200
Processing row_index: 1300
Begin current loop: 1300
Processing row_index: 1400
Begin current loop: 1400
Processing row_index: 1500
Begin current loop: 1500
Processing row_index: 1600
Begin current loop: 1600
Processing row_index: 1700
Begin current loop: 1700
Processing row_index: 1800
Begin current loop: 1800
Processing row_index: 1900
Begin current loop: 1900
Processing row_index: 2000
Beg

#### 3.6.2 Fine tune model

In [ ]:
max_length = 128

train_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(es_en_train),
    data_filename=train_file,
    max_length=max_length
)

valid_data_iterator = TranslationDataIterator(
    tokenizer=m2m_tokenizer,
    n_examples=len(es_en_val),
    data_filename=valid_file,
    max_length=max_length
)

In [ ]:
srg_lang = 'es'
tgt_lang = 'en'

# Modify this filepath to where you want to save the model after fine-tuning
dir_path = 'drive/MyDrive/W266/es_en/new_model/'
file_path = dir_path + f'M2M100_news_commentary_{srg_lang}_{tgt_lang}'

In [ ]:
# Specify batch size and other training arguments

batch_size = 8

args = Seq2SeqTrainingArguments(
    file_path,
    evaluation_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Define the trainer, passing in the model, training args, and data generators

trainer = Seq2SeqTrainer(
    m2m_model,
    args,
    train_dataset=train_data_iterator,
    eval_dataset=valid_data_iterator
)

In [ ]:
# Call train

trainer.train()

***** Running training *****
  Num examples = 119436
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 14930
  Number of trainable parameters = 483905536


Epoch,Training Loss,Validation Loss
1,0.243700,0.230385


Saving model checkpoint to drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-500
Configuration saved in drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-500/config.json
Model weights saved in drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-500/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-1000
Configuration saved in drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-1000/config.json
Model weights saved in drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-1500
Configuration saved in drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoint-1500/config.json
Model weights saved in drive/MyDrive/W266/es_en/new_model/M2M100_news_commentary_es_en/checkpoi

TrainOutput(global_step=14930, training_loss=0.2690360946297566, metrics={'train_runtime': 20506.9924, 'train_samples_per_second': 5.824, 'train_steps_per_second': 0.728, 'total_flos': 3.2472978910347264e+16, 'train_loss': 0.2690360946297566, 'epoch': 1.0})

In [ ]:
trainer.save_model('drive/MyDrive/W266/M2M100_news_commentary_es_en/checkpoint-001')

Saving model checkpoint to drive/MyDrive/W266/M2M100_news_commentary_es_en/checkpoint-001
Configuration saved in drive/MyDrive/W266/M2M100_news_commentary_es_en/checkpoint-001/config.json
Model weights saved in drive/MyDrive/W266/M2M100_news_commentary_es_en/checkpoint-001/pytorch_model.bin


In [ ]:
m2m_model_saved = M2M100ForConditionalGeneration.from_pretrained('drive/MyDrive/W266/M2M100_news_commentary_es_en/checkpoint-001')

loading configuration file drive/MyDrive/W266/M2M100_news_commentary_es_en/checkpoint-001/config.json
Model config M2M100Config {
  "_name_or_path": "facebook/m2m100_418M",
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "architectures": [
    "M2M100ForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.05,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.05,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_length": 200,
  "max_position_embeddings": 1024,
  "model_type": "m2m_100",
  "num_beams": 5,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "scale_embedding": true,
  "torch_dtype": "float32",
  "transformers_version

In [ ]:
df_valid_list = df_valid[f'{srg_lang}'].tolist()
df_valid_label= df_valid[f'{tgt_lang}'].tolist()

In [ ]:
df_valid_label[:3]

['The only notable exception, Justice Minister Tzipi Livni, seems to have been appointed merely to provide a veneer of official commitment to the peace process.',
 'At around 47% of GDP, China’s embryonic services sector remains well short of the 60-65% share that a middle-income economy typically possesses.',
 'Whatever weaknesses one finds in the Reinhart/Rogoff paper, it does not follow that countries in recession should always disregard deficits and debt levels and focus on stimulus.']

In [ ]:
eval_score = []
step = 100


for row_index in range(0,len(df_valid_list),step):

  print(f"Processing row_index: {row_index}")

  if os.path.isfile(f"drive/MyDrive/W266/es_en/new_model/es_en_validation_{row_index}.csv"):
    print(f"  row_index {row_index} exist, continue...")
    continue
  else:

    curr_input = df_valid_list[row_index: row_index + step]
    curr_label = df_valid_label[row_index: row_index + step]


    # curr_loop = row_index/step
    print(f"  Processing row_index: {row_index}")

    encoded_en = m2m_tokenizer(curr_input, 
                          padding = True,
                          max_length=50,
                          truncation=True,
                          return_tensors='pt')
    generated_tokens = m2m_model_saved.generate(**encoded_en, forced_bos_token_id=m2m_tokenizer.get_lang_id(f'{tgt_lang}'))
    curr_output = m2m_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    ### Compute BLEURT score
    bleurt_label = [[label] for label in curr_label]
    curr_bleurt_score = bleurt_metric.compute(predictions=curr_output, references=bleurt_label)

    eval_score.append(curr_bleurt_score['scores'])

    
    df_output = pd.DataFrame(eval_score)
    df_output.to_csv(f'drive/MyDrive/W266/es_en/new_model/es_en_validation_{row_index}.csv', sep=',', index=False)
  

Processing row_index: 0
  Processing row_index: 0
Processing row_index: 100
  Processing row_index: 100
Processing row_index: 200
  Processing row_index: 200
Processing row_index: 300
  Processing row_index: 300
Processing row_index: 400
  Processing row_index: 400
Processing row_index: 500
  Processing row_index: 500
Processing row_index: 600
  Processing row_index: 600
Processing row_index: 700
  Processing row_index: 700
Processing row_index: 800
  Processing row_index: 800
Processing row_index: 900
  Processing row_index: 900
Processing row_index: 1000
  Processing row_index: 1000
Processing row_index: 1100
  Processing row_index: 1100
Processing row_index: 1200
  Processing row_index: 1200
Processing row_index: 1300
  Processing row_index: 1300
Processing row_index: 1400
  Processing row_index: 1400
Processing row_index: 1500
  Processing row_index: 1500
Processing row_index: 1600
  Processing row_index: 1600
Processing row_index: 1700
  Processing row_index: 1700
Processing row_i